In [2]:
import pandas as pd
import glob
import os
from pprint import pprint

In [4]:
#Get all CSV files in the folder of GA
all_files = glob.glob(r"C:\Huy Phan\College\VoterTurnout\data\RI\*.csv")

# Files that contain both 'precinct' and 'general' in the filename
general_files = [
    f for f in all_files
    if all(word in f.lower() for word in ['precinct', 'general'])
]

# Files that contain both 'precinct' and 'primary' in the filename
primary_files = [
    f for f in all_files
    if all(word in f.lower() for word in ['precinct', 'primary', 'president'])
]


In [5]:
print("General files:")
for f in general_files:
    print(f)


General files:
C:\Huy Phan\College\VoterTurnout\data\RI\20161108__ri__general__precinct.csv


In [6]:
print("\nPrimary files:")
for f in primary_files:
    print(f)


Primary files:
C:\Huy Phan\College\VoterTurnout\data\RI\20160426__ri__primary__president__precinct.csv


In [14]:
# Process primary files
primary_df_list = []

for file in primary_files:
    try:
        df = pd.read_csv(file)

        # Drop rows where 'precinct' is NaN
        if 'office' in df.columns:
            df = df[df["office"] == "President"]
        
        df = df.drop_duplicates()

        primary_df_list.append(df)

    except Exception as e:
        print(f"Error in {file}: {e}")

# Combine all cleaned files
pri_combined_df = pd.concat(primary_df_list, ignore_index=True)
# Create a new column "precinct" by combining county and precinct
pri_combined_df


,county,precinct,office,district,party,candidate,votes
0,Bristol,Barrington 0101,President,NaN,Democratic,Mark Stewart,0
1,Bristol,Barrington 0101,President,NaN,Democratic,Bernie Sanders,251
2,Bristol,Barrington 0101,President,NaN,Democratic,Uncommitted,5
3,Bristol,Barrington 0101,President,NaN,Democratic,"Roque ""Rocky"" De La Fuente",0
4,Bristol,Barrington 0101,President,NaN,Democratic,Hillary Clinton,214
...,...,...,...,...,...,...,...
2039,NaN,Federal Precinct #2,President,NaN,Republican,John R. Kasich,249
2040,NaN,Federal Precinct #2,President,NaN,Republican,Marco Rubio,11
2041,NaN,Federal Precinct #2,President,NaN,Republican,Uncommitted,25
2042,NaN,Federal Precinct #2,President,NaN,Republican,Overvotes,2


In [15]:
pri_combined_df["party"].value_counts(dropna=False)

party
Democratic    1022
Republican    1022
Name: count, dtype: int64

In [22]:
# Select only the relevant columns
primary_data = pri_combined_df[["precinct", "party", "candidate", "votes"]]
primary_data.loc[:, "party"] = primary_data["party"].replace({
    "Democratic": "DEM",
    "Republican": "REP"
})
# primary_data = primary_data[primary_data["party"].isin(["DEM", "REP"])] # Analyzing only republican and democratic

primary_data

,precinct,party,candidate,votes
0,Barrington 0101,DEM,Mark Stewart,0
1,Barrington 0101,DEM,Bernie Sanders,251
2,Barrington 0101,DEM,Uncommitted,5
3,Barrington 0101,DEM,"Roque ""Rocky"" De La Fuente",0
4,Barrington 0101,DEM,Hillary Clinton,214
...,...,...,...,...
2039,Federal Precinct #2,REP,John R. Kasich,249
2040,Federal Precinct #2,REP,Marco Rubio,11
2041,Federal Precinct #2,REP,Uncommitted,25
2042,Federal Precinct #2,REP,Overvotes,2


In [23]:
primary_data["party"].value_counts(dropna=False)

party
DEM    1022
REP    1022
Name: count, dtype: int64

In [26]:
#Remove Uncommited, Overvotes, Undervotes
primary_data = primary_data[~primary_data["candidate"].isin(["Uncommitted", "Overvotes", "Undervotes"])] 
primary_data["candidate"].value_counts(dropna=False)

candidate
Mark Stewart                  146
Bernie Sanders                146
Roque "Rocky" De La Fuente    146
Hillary Clinton               146
Donald J. Trump               146
Ted Cruz                      146
John R. Kasich                146
Marco Rubio                   146
Name: count, dtype: int64

In [27]:
primary_data["candidate"].unique()
candidate_party_map = (
    primary_data.dropna(subset=["candidate", "party"])
                .set_index("candidate")["party"]
                .to_dict()
)
print(candidate_party_map)

{'Mark Stewart': 'DEM', 'Bernie Sanders': 'DEM', 'Roque "Rocky" De La Fuente': 'DEM', 'Hillary Clinton': 'DEM', 'Donald J. Trump': 'REP', 'Ted Cruz': 'REP', 'John R. Kasich': 'REP', 'Marco Rubio': 'REP'}


In [28]:
primary_data.loc[:,"candidate_column"] = (
    "pri_" +
    primary_data["party"].str.lower() + "_" +
    primary_data["candidate"].str.split().str[-1].str.upper()
)

# pivot the table
primary_result = primary_data.pivot_table(
    index="precinct",
    columns="candidate_column",
    values="votes",
    aggfunc="sum",  
    fill_value=0
).reset_index()

primary_result

candidate_column,precinct,pri_dem_CLINTON,pri_dem_FUENTE,pri_dem_SANDERS,pri_dem_STEWART,pri_rep_CRUZ,pri_rep_KASICH,pri_rep_RUBIO,pri_rep_TRUMP
0,Barrington 0101,214,0,251,0,34,92,2,115
1,Barrington 0102,336,0,347,0,29,119,2,138
2,Barrington 0103,313,0,216,0,25,153,4,124
3,Barrington 0104,316,1,299,0,23,158,3,173
4,Barrington 0105,311,1,286,1,33,178,3,137
...,...,...,...,...,...,...,...,...,...
141,Woonsocket 3902,159,0,105,1,5,7,4,62
142,Woonsocket 3903,337,1,394,2,39,31,4,181
143,Woonsocket 3905,124,0,164,1,14,10,0,67
144,Woonsocket 3907,339,2,488,0,49,62,2,313


In [30]:
# Process general files
gen_df_list = []

for file in general_files:
    try:
        df = pd.read_csv(file)

        # Select only president
        if 'office' in df.columns:
            df = df[df["office"] == "President" ]

        gen_df_list.append(df)

    except Exception as e:
        print(f"Error in {file}: {e}")

# Combine all cleaned files
gen_combined_df = pd.concat(gen_df_list, ignore_index=True)
gen_combined_df


,office,district,candidate,party,precinct,votes,absentee_votes
0,President,NaN,Donald J. Trump,R,Barrington 0101,522,36
1,President,NaN,Donald J. Trump,R,Barrington 0102,683,67
2,President,NaN,Donald J. Trump,R,Barrington 0103,469,39
3,President,NaN,Donald J. Trump,R,Barrington 0104,611,38
4,President,NaN,Donald J. Trump,R,Barrington 0105,590,68
...,...,...,...,...,...,...,...
3001,President,NaN,WRITE-IN,NaN,Woonsocket 3912,17,0
3002,President,NaN,WRITE-IN,NaN,Woonsocket Limited,0,0
3003,President,NaN,WRITE-IN,NaN,Woonsocket Presidential,4,0
3004,President,NaN,WRITE-IN,NaN,Federal Precinct #1,1,1


In [32]:
general_data = gen_combined_df[["precinct", "party", "candidate", "votes"]]
general_data["party"].value_counts(dropna=False)


party
NaN    1002
R       501
D       501
L       501
G       501
Name: count, dtype: int64

In [33]:
general_data["candidate"].value_counts(dropna=False)

candidate
Donald J. Trump               501
Hillary Clinton               501
Gary Johnson                  501
Jill Stein                    501
"Rocky" Roque De La Fuente    501
WRITE-IN                      501
Name: count, dtype: int64

In [35]:
# remove WRITE-IN
general_data = general_data[~general_data["candidate"].isin(["WRITE-IN"])] 
general_data["candidate"].value_counts(dropna=False)

candidate
Donald J. Trump               501
Hillary Clinton               501
Gary Johnson                  501
Jill Stein                    501
"Rocky" Roque De La Fuente    501
Name: count, dtype: int64

In [36]:
# general_data = general_data[~general_data["candidate"].isin(["YES", "NO"])]
# general_data["party"] = general_data.apply(
#     lambda row: candidate_party_map.get(row["candidate"], row["party"]) if pd.isna(row["party"]) else row["party"],
#     axis=1
# )
general_data.loc[:, "party"] = general_data["party"].replace({
    "D": "DEM",
    "R": "REP"
})
general_data = general_data[general_data["party"].isin(["DEM", "REP"])] # Analyzing only republican and democratic
general_data

,precinct,party,candidate,votes
0,Barrington 0101,REP,Donald J. Trump,522
1,Barrington 0102,REP,Donald J. Trump,683
2,Barrington 0103,REP,Donald J. Trump,469
3,Barrington 0104,REP,Donald J. Trump,611
4,Barrington 0105,REP,Donald J. Trump,590
...,...,...,...,...
997,Woonsocket 3912,DEM,Hillary Clinton,506
998,Woonsocket Limited,DEM,Hillary Clinton,0
999,Woonsocket Presidential,DEM,Hillary Clinton,104
1000,Federal Precinct #1,DEM,Hillary Clinton,376


In [38]:
general_data["candidate_column"] = (
    "gen_" +
    general_data["party"].str.lower() + "_" +
    general_data["candidate"].str.split().str[-1].str.upper()
)

# pivot the table
general_result = general_data.pivot_table(
    index="precinct",
    columns="candidate_column",
    values="votes",
    aggfunc="sum",  
    fill_value=0
).reset_index()

general_result

candidate_column,precinct,gen_dem_CLINTON,gen_rep_TRUMP
0,Barrington 0101,1027,522
1,Barrington 0102,1331,683
2,Barrington 0103,1136,469
3,Barrington 0104,1328,611
4,Barrington 0105,1290,590
...,...,...,...
496,Woonsocket 3910,686,759
497,Woonsocket 3911,677,465
498,Woonsocket 3912,506,448
499,Woonsocket Limited,0,0


In [39]:
combined = pd.merge(primary_result, general_result, on="precinct", how="inner")
combined["rep_primary_total"] = combined.filter(like="pri_rep_").sum(axis=1)
combined["dem_primary_total"] = combined.filter(like="pri_dem_").sum(axis=1)
combined["general_total"] = combined.filter(like="gen_").sum(axis=1)
combined

candidate_column,precinct,pri_dem_CLINTON,pri_dem_FUENTE,pri_dem_SANDERS,pri_dem_STEWART,pri_rep_CRUZ,pri_rep_KASICH,pri_rep_RUBIO,pri_rep_TRUMP,gen_dem_CLINTON,gen_rep_TRUMP,rep_primary_total,dem_primary_total,general_total
0,Barrington 0101,214,0,251,0,34,92,2,115,1027,522,243,465,1549
1,Barrington 0102,336,0,347,0,29,119,2,138,1331,683,288,683,2014
2,Barrington 0103,313,0,216,0,25,153,4,124,1136,469,306,529,1605
3,Barrington 0104,316,1,299,0,23,158,3,173,1328,611,357,616,1939
4,Barrington 0105,311,1,286,1,33,178,3,137,1290,590,351,599,1880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,Woonsocket 3902,159,0,105,1,5,7,4,62,400,230,78,265,630
142,Woonsocket 3903,337,1,394,2,39,31,4,181,300,162,255,734,462
143,Woonsocket 3905,124,0,164,1,14,10,0,67,506,425,91,289,931
144,Woonsocket 3907,339,2,488,0,49,62,2,313,563,628,426,829,1191


In [40]:
combined.to_csv("RI.csv", index=False)
